In [ ]:
import tkinter as tk
import numpy as np
import random
import time
from queue import Queue, PriorityQueue
from collections import deque

# Initialize global variables
GRID_SIZE = 20
CANVAS_SIZE = 600
BUTTON_HEIGHT = 50

# Maze sizes for small, medium, and large
SMALL_MAZE = 10
MEDIUM_MAZE = 20
LARGE_MAZE = 30

# 8-Puzzle Goal State
PUZZLE_GOAL_STATE = [1, 2, 3, 4, 5, 6, 7, 8, 0]  # 0 represents the empty tile

class MazeSolverApp:
    def __init__(self, root):
        self.root = root
        self.root.title("AI Search Problem Solver")

        # Create canvas for problems
        self.canvas = tk.Canvas(self.root, width=CANVAS_SIZE, height=CANVAS_SIZE, bg='white')
        self.canvas.grid(row=0, column=0, columnspan=6)

        # Control buttons
        self.random_button = tk.Button(self.root, text="Random Maze", command=self.generate_random_maze)
        self.random_button.grid(row=1, column=0)

        self.dfs_button = tk.Button(self.root, text="Solve with DFS", command=self.solve_with_dfs)
        self.dfs_button.grid(row=1, column=1)

        self.bfs_button = tk.Button(self.root, text="Solve with BFS", command=self.solve_with_bfs)
        self.bfs_button.grid(row=1, column=2)

        self.astar_button = tk.Button(self.root, text="Solve with A*", command=self.solve_with_astar)
        self.astar_button.grid(row=1, column=3)

        self.bidirectional_button = tk.Button(self.root, text="Solve with Bidirectional", command=self.solve_with_bidirectional)
        self.bidirectional_button.grid(row=1, column=4)

        self.problem_var = tk.StringVar(value="Maze")
        self.problem_menu = tk.OptionMenu(self.root, self.problem_var, "Maze", "String Transformation", "8-Puzzle", command=self.change_problem)
        self.problem_menu.grid(row=1, column=5)

        # Set the default problem type and initialize
        self.problem_type = "Maze"
        self.solution_path = None
        self.explored_path = []

        self.generate_random_maze()

    def change_problem(self, problem):
        self.problem_type = problem
        if problem == "Maze":
            self.generate_random_maze()
        elif problem == "String Transformation":
            self.generate_string_problem()
        elif problem == "8-Puzzle":
            self.generate_puzzle_problem()

    # Maze Problem
    def generate_random_maze(self):
        self.maze_size = SMALL_MAZE
        self.maze = self.generate_maze(self.maze_size)
        self.draw_maze()

    def generate_maze(self, size):
        maze = np.ones((size, size), dtype=int)
        maze[0][0] = 0
        maze[size - 1][size - 1] = 0

        def dfs_generate(x, y):
            directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
            random.shuffle(directions)

            for dx, dy in directions:
                nx, ny = x + dx, y + dy
                if 0 <= nx < size and 0 <= ny < size and maze[nx][ny] == 1:
                    maze[nx][ny] = 0
                    dfs_generate(nx, ny)

        dfs_generate(0, 0)
        return maze

    def draw_maze(self):
        self.canvas.delete("all")
        size = len(self.maze)
        cell_size = CANVAS_SIZE // size
        for i in range(size):
            for j in range(size):
                color = 'white' if self.maze[i][j] == 0 else 'black'
                if (i, j) == (0, 0):
                    color = 'green'
                elif (i, j) == (size - 1, size - 1):
                    color = 'red'
                self.canvas.create_rectangle(j * cell_size, i * cell_size,
                                             (j + 1) * cell_size, (i + 1) * cell_size, fill=color)

    # String Transformation Problem
    def generate_string_problem(self):
        self.start_word = "cat"
        self.end_word = "dog"
        self.valid_words = {"cat", "bat", "bot", "bog", "dog", "cog", "cot", "dot"}
        self.solution_path = []
        self.display_string_problem()

    def display_string_problem(self):
        self.canvas.delete("all")
        self.canvas.create_text(CANVAS_SIZE / 2, CANVAS_SIZE / 3, text=f"Transform '{self.start_word}' to '{self.end_word}'", font=("Arial", 20))

    # 8-Puzzle Problem
    def generate_puzzle_problem(self):
        self.puzzle_start = [1, 2, 3, 4, 5, 6, 0, 7, 8]
        self.puzzle_goal = PUZZLE_GOAL_STATE
        self.solution_path = []
        self.display_puzzle()

    def display_puzzle(self):
        self.canvas.delete("all")
        size = int(len(self.puzzle_start) ** 0.5)
        cell_size = CANVAS_SIZE // size
        for i in range(size):
            for j in range(size):
                value = self.puzzle_start[i * size + j]
                color = 'white' if value != 0 else 'black'
                self.canvas.create_rectangle(j * cell_size, i * cell_size,
                                             (j + 1) * cell_size, (i + 1) * cell_size, fill=color)
                if value != 0:
                    self.canvas.create_text(j * cell_size + cell_size / 2, i * cell_size + cell_size / 2,
                                            text=str(value), font=("Arial", 20))

    def solve_with_dfs(self):
        self.solve_problem(dfs, "DFS")

    def solve_with_bfs(self):
        self.solve_problem(bfs, "BFS")

    def solve_with_astar(self):
        self.solve_problem(a_star, "A*")

    def solve_with_bidirectional(self):
        self.solve_problem(bidirectional_search, "Bidirectional Search")

    def solve_problem(self, algorithm, name):
        start_time = time.time()
        if self.problem_type == "Maze":
            self.explored_path, self.solution_path = algorithm(self.maze)
        elif self.problem_type == "String Transformation":
            self.solution_path = algorithm(self.start_word, self.end_word, self.valid_words)
        elif self.problem_type == "8-Puzzle":
            self.solution_path = algorithm(self.puzzle_start, self.puzzle_goal)
        elapsed_time = time.time() - start_time
        print(f"{name} solved the problem in {elapsed_time:.4f} seconds.")
        
        if self.solution_path:
            self.animate_solution()
        else:
            self.show_error_message("No solution found!")

    def animate_solution(self):
        if self.problem_type == "Maze":
            self.animate_maze_solution()
        elif self.problem_type == "String Transformation":
            self.animate_string_solution()
        elif self.problem_type == "8-Puzzle":
            self.animate_puzzle_solution()

    def animate_maze_solution(self):
        size = len(self.maze)
        cell_size = CANVAS_SIZE // size
        for i, (x, y) in enumerate(self.solution_path):
            self.canvas.create_rectangle(y * cell_size, x * cell_size,
                                         (y + 1) * cell_size, (x + 1) * cell_size, fill='yellow')
            if i > 0:
                prev_x, prev_y = self.solution_path[i-1]
                self.canvas.create_line((prev_y + 0.5) * cell_size, (prev_x + 0.5) * cell_size,
                                        (y + 0.5) * cell_size, (x + 0.5) * cell_size, fill='red', width=2)
            self.root.update()
            time.sleep(0.1)

    def animate_string_solution(self):
        y_pos = CANVAS_SIZE // 2
        for i, word in enumerate(self.solution_path):
            self.canvas.delete("all")
            self.canvas.create_text(CANVAS_SIZE // 2, y_pos, text=word, font=("Arial", 20))
            if i > 0:
                prev_word = self.solution_path[i-1]
                diff_index = next(j for j in range(len(word)) if word[j] != prev_word[j])
                self.canvas.create_text(CANVAS_SIZE // 2, y_pos + 30, 
                                        text=f"Changed '{prev_word[diff_index]}' to '{word[diff_index]}'",
                                        font=("Arial", 16))
            self.root.update()
            time.sleep(0.5)

    def animate_puzzle_solution(self):
        size = int(len(self.puzzle_start) ** 0.5)
        cell_size = CANVAS_SIZE // size
        for state in self.solution_path:
            self.canvas.delete("all")
            for i in range(size):
                for j in range(size):
                    value = state[i * size + j]
                    color = 'white' if value != 0 else 'black'
                    self.canvas.create_rectangle(j * cell_size, i * cell_size,
                                                 (j + 1) * cell_size, (i + 1) * cell_size, fill=color)
                    if value != 0:
                        self.canvas.create_text(j * cell_size + cell_size / 2, i * cell_size + cell_size / 2,
                                                text=str(value), font=("Arial", 20))
            self.root.update()
            time.sleep(0.5)

    def show_error_message(self, message):
        self.canvas.delete("all")
        self.canvas.create_text(CANVAS_SIZE / 2, CANVAS_SIZE / 2,
                                text=message, font=("Arial", 20), fill='red')
        self.root.update()
        time.sleep(2)

# Maze solving algorithms
def dfs(maze):
    start = (0, 0)
    goal = (len(maze) - 1, len(maze) - 1)
    stack = [(start, [start])]
    visited = set()

    while stack:
        (x, y), path = stack.pop()
        if (x, y) == goal:
            return visited, path
        if (x, y) not in visited:
            visited.add((x, y))
            for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
                nx, ny = x + dx, y + dy
                if 0 <= nx < len(maze) and 0 <= ny < len(maze) and maze[nx][ny] == 0:
                    stack.append(((nx, ny), path + [(nx, ny)]))
    return visited, None

def bfs(maze):
    start = (0, 0)
    goal = (len(maze) - 1, len(maze) - 1)
    queue = Queue()
    queue.put((start, [start]))
    visited = set()

    while not queue.empty():
        (x, y), path = queue.get()
        if (x, y) == goal:
            return visited, path
        if (x, y) not in visited:
            visited.add((x, y))
            for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
                nx, ny = x + dx, y + dy
                if 0 <= nx < len(maze) and 0 <= ny < len(maze) and maze[nx][ny] == 0:
                    queue.put(((nx, ny), path + [(nx, ny)]))
    return visited, None

def manhattan_distance(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def a_star(maze):
    start = (0, 0)
    goal = (len(maze) - 1, len(maze) - 1)
    open_set = PriorityQueue()
    open_set.put((0, start, [start]))
    g_score = {start: 0}
    f_score = {start: manhattan_distance(start, goal)}
    visited = set()

    while not open_set.empty():
        _, current, path = open_set.get()
        if current == goal:
            return visited, path
        if current not in visited:
            visited.add(current)
            for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
                nx, ny = current[0] + dx, current[1] + dy
                if 0 <= nx < len(maze) and 0 <= ny < len(maze) and maze[nx][ny] == 0:
                    neighbor = (nx, ny)
                    tentative_g_score = g_score[current] + 1
                    if tentative_g_score < g_score.get(neighbor, float('inf')):
                        g_score[neighbor] = tentative_g_score
                        f_score[neighbor] = tentative_g_score + manhattan_distance(neighbor, goal)
                        open_set.put((f_score[neighbor], neighbor, path + [neighbor]))
    return visited, None

def bidirectional_search(maze):
    start = (0, 0)
    goal = (len(maze) - 1, len(maze) - 1)
    forward_queue = Queue()
    backward_queue = Queue()
    forward_queue.put((start, [start]))
    backward_queue.put((goal, [goal]))
    forward_visited = {start: [start]}
    backward_visited = {goal: [goal]}

    while not forward_queue.empty() and not backward_queue.empty():
        # Forward search
        current, path = forward_queue.get()
        for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            nx, ny = current[0] + dx, current[1] + dy
            if 0 <= nx < len(maze) and 0 <= ny < len(maze) and maze[nx][ny] == 0:
                next_node = (nx, ny)
                if next_node in backward_visited:
                    return set(forward_visited.keys()) | set(backward_visited.keys()), path + backward_visited[next_node][::-1]
                if next_node not in forward_visited:
                    forward_visited[next_node] = path + [next_node]
                    forward_queue.put((next_node, path + [next_node]))

        # Backward search
        current, path = backward_queue.get()
        for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            nx, ny = current[0] + dx, current[1] + dy
            if 0 <= nx < len(maze) and 0 <= ny < len(maze) and maze[nx][ny] == 0:
                next_node = (nx, ny)
                if next_node in forward_visited:
                    return set(forward_visited.keys()) | set(backward_visited.keys()), forward_visited[next_node] + path[::-1]
                if next_node not in backward_visited:
                    backward_visited[next_node] = [next_node] + path
                    backward_queue.put((next_node, [next_node] + path))

    return set(), None

# String Transformation algorithms
def bfs_string(start, end, valid_words):
    queue = Queue()
    queue.put([start])
    visited = set([start])

    while not queue.empty():
        path = queue.get()
        word = path[-1]

        if word == end:
            return path

        for i in range(len(word)):
            for c in 'abcdefghijklmnopqrstuvwxyz':
                new_word = word[:i] + c + word[i+1:]
                if new_word in valid_words and new_word not in visited:
                    visited.add(new_word)
                    new_path = path + [new_word]
                    queue.put(new_path)
                    if new_word == end:
                        return new_path

    return None

def dfs_string(start, end, valid_words):
    stack = [[start]]
    visited = set([start])

    while stack:
        path = stack.pop()
        word = path[-1]

        if word == end:
            return path

        for i in range(len(word)):
            for c in 'abcdefghijklmnopqrstuvwxyz':
                new_word = word[:i] + c + word[i+1:]
                if new_word in valid_words and new_word not in visited:
                    visited.add(new_word)
                    new_path = path + [new_word]
                    stack.append(new_path)
                    if new_word == end:
                        return new_path

    return None

def a_star_string(start, end, valid_words):
    def heuristic(word1, word2):
        return sum(a != b for a, b in zip(word1, word2))

    open_set = PriorityQueue()
    open_set.put((0, [start]))
    g_score = {start: 0}
    f_score = {start: heuristic(start, end)}

    while not open_set.empty():
        _, path = open_set.get()
        word = path[-1]

        if word == end:
            return path

        for i in range(len(word)):
            for c in 'abcdefghijklmnopqrstuvwxyz':
                new_word = word[:i] + c + word[i+1:]
                if new_word in valid_words:
                    tentative_g_score = g_score[word] + 1
                    if tentative_g_score < g_score.get(new_word, float('inf')):
                        new_path = path + [new_word]
                        g_score[new_word] = tentative_g_score
                        f_score[new_word] = tentative_g_score + heuristic(new_word, end)
                        open_set.put((f_score[new_word], new_path))

    return None

def bidirectional_search_string(start, end, valid_words):
    forward_queue = Queue()
    backward_queue = Queue()
    forward_queue.put([start])
    backward_queue.put([end])
    forward_visited = {start: [start]}
    backward_visited = {end: [end]}

    while not forward_queue.empty() and not backward_queue.empty():
        # Forward search
        path = forward_queue.get()
        word = path[-1]

        for i in range(len(word)):
            for c in 'abcdefghijklmnopqrstuvwxyz':
                new_word = word[:i] + c + word[i+1:]
                if new_word in valid_words:
                    if new_word in backward_visited:
                        return path + backward_visited[new_word][::-1][1:]
                    if new_word not in forward_visited:
                        new_path = path + [new_word]
                        forward_visited[new_word] = new_path
                        forward_queue.put(new_path)

        # Backward search
        path = backward_queue.get()
        word = path[-1]

        for i in range(len(word)):
            for c in 'abcdefghijklmnopqrstuvwxyz':
                new_word = word[:i] + c + word[i+1:]
                if new_word in valid_words:
                    if new_word in forward_visited:
                        return forward_visited[new_word] + path[::-1][1:]
                    if new_word not in backward_visited:
                        new_path = [new_word] + path
                        backward_visited[new_word] = new_path
                        backward_queue.put(new_path)

    return None

# 8-Puzzle algorithms
def get_neighbors(state):
    neighbors = []
    empty_index = state.index(0)
    row, col = empty_index // 3, empty_index % 3

    for dr, dc in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
        new_row, new_col = row + dr, col + dc
        if 0 <= new_row < 3 and 0 <= new_col < 3:
            new_state = state[:]
            new_index = new_row * 3 + new_col
            new_state[empty_index], new_state[new_index] = new_state[new_index], new_state[empty_index]
            neighbors.append(new_state)

    return neighbors

def bfs_puzzle(start, goal):
    queue = Queue()
    queue.put((start, [start]))
    visited = set(tuple(start))

    while not queue.empty():
        state, path = queue.get()

        if state == goal:
            return path

        for neighbor in get_neighbors(state):
            if tuple(neighbor) not in visited:
                visited.add(tuple(neighbor))
                queue.put((neighbor, path + [neighbor]))

    return None

def dfs_puzzle(start, goal):
    stack = [(start, [start])]
    visited = set(tuple(start))

    while stack:
        state, path = stack.pop()

        if state == goal:
            return path

        for neighbor in get_neighbors(state):
            if tuple(neighbor) not in visited:
                visited.add(tuple(neighbor))
                stack.append((neighbor, path + [neighbor]))

    return None

def manhattan_distance_puzzle(state, goal):
    distance = 0
    for i in range(9):
        if state[i] != 0 and state[i] != goal[i]:
            current_row, current_col = i // 3, i % 3
            goal_index = goal.index(state[i])
            goal_row, goal_col = goal_index // 3, goal_index % 3
            distance += abs(current_row - goal_row) + abs(current_col - goal_col)
    return distance

def a_star_puzzle(start, goal):
    open_set = PriorityQueue()
    open_set.put((0, start, [start]))
    g_score = {tuple(start): 0}
    f_score = {tuple(start): manhattan_distance_puzzle(start, goal)}

    while not open_set.empty():
        _, state, path = open_set.get()

        if state == goal:
            return path

        for neighbor in get_neighbors(state):
            tentative_g_score = g_score[tuple(state)] + 1
            if tentative_g_score < g_score.get(tuple(neighbor), float('inf')):
                new_path = path + [neighbor]
                g_score[tuple(neighbor)] = tentative_g_score
                f_score[tuple(neighbor)] = tentative_g_score + manhattan_distance_puzzle(neighbor, goal)
                open_set.put((f_score[tuple(neighbor)], neighbor, new_path))

    return None

def bidirectional_search_puzzle(start, goal):
    forward_queue = Queue()
    backward_queue = Queue()
    forward_queue.put((start, [start]))
    backward_queue.put((goal, [goal]))
    forward_visited = {tuple(start): [start]}
    backward_visited = {tuple(goal): [goal]}

    while not forward_queue.empty() and not backward_queue.empty():
        # Forward search
        state, path = forward_queue.get()

        for neighbor in get_neighbors(state):
            if tuple(neighbor) in backward_visited:
                return path + backward_visited[tuple(neighbor)][::-1][1:]
            if tuple(neighbor) not in forward_visited:
                new_path = path + [neighbor]
                forward_visited[tuple(neighbor)] = new_path
                forward_queue.put((neighbor, new_path))

        # Backward search
        state, path = backward_queue.get()

        for neighbor in get_neighbors(state):
            if tuple(neighbor) in forward_visited:
                return forward_visited[tuple(neighbor)] + path[::-1][1:]
            if tuple(neighbor) not in backward_visited:
                new_path = [neighbor] + path
                backward_visited[tuple(neighbor)] = new_path
                backward_queue.put((neighbor, new_path))

    return None

# Main execution
if __name__ == "__main__":
    root = tk.Tk()
    app = MazeSolverApp(root)
    root.mainloop()